# Jinja2 Templating for YAML / YAMA AP

This first cell is only required to install dependecies locally. Mybinder installs the dependecies from `requirements.txt`, so no need to run this cell in MyBinder.org

In [ ]:
!pip install PyShExC
!pip install Jinja2
!pip install ruamel.yaml[jinja2]

### 1. MAP in YAML

In [ ]:
ap_yaml = """

%YAML 1.1
---

namespaces:
  dc: http://purl.org/dc/terms/
  bibframe: http://id.loc.gov/ontologies/bibframe/
  foaf: http://xmlns.com/foaf/0.1/
  schema: http://schema.org/
  xsd: http://www.w3.org/2001/XMLSchema#

value:

  - &value_literal_3_250
    valueID: value_literal_3_250
    dataType: xsd:string 
    minLength: 3 
    maxLength: 250
  
  - &any_name
    valueID: any_name
    dataType: xsd:string
    
  - &url
    valueID: url
    dataType: xsd:anyURI

statements:

  - &title
    statementID: title 
    displayName: Title
    property: dc:title
    type: literal
    value : *value_literal_3_250

  - &name
    statementID: name 
    displayName: Name
    property: foaf:name
    type: literal
    value: *any_name
  
  - &homepage
    statementID: homepage
    displayName: Homepage
    property: foaf:homepage
    type: IRI
    value: *url

  - &organization
    statementID: organization 
    displayName: Organization
    property: foaf:Organization
    type: literal
    value : *any_name


descriptions:

  - &book
    descID: book 
    displayName: Book
    property: schema:Book
    statements: 
      - << : *title
        minOccur : 1
        maxOccur : 1

  - &creator
    descID: creator 
    displayName: Creator 
    property: dc:creator
    statements: 
      - << : *name
        minOccur : 1
        maxOccur : -1
      - << : *homepage
        minOccur : 0
        maxOccur : 1

  - &publisher
    descID: publisher 
    displayName: publisher
    property: dc:publisher
    statements: 
      - << : *organization
        minOccur : 1
        maxOccur : 1
      - << : *homepage
        minOccur : 0
        maxOccur : 1

  - &distributor
    descID: distributor 
    displayName: Distributor
    property: bibframe:Distribution
    notes: >
        Online and offline distributors can be included. 
        For the case of online distributors, 
        also, add the storefront as a homepage.
    statements: 
      - << : *organization
        minOccur : 1
        maxOccur : -1
      - << : *homepage
        minOccur : 0
        maxOccur : -1

descriptionSet: 

  descSetID: BookCase
  title: The Book Case
  notes: The curios case of a book as an application profile
  descriptions : [*book, *creator, *publisher, *distributor]


"""

### 2. Parsing YAML

In [ ]:
from ruamel.yaml import YAML

yaml=YAML()
ap=yaml.load(ap_yaml)

### 3. Parsed AP structure

In [ ]:
import json
print(json.dumps(ap, indent=4))

### 4. ShExC Jinja2 Template

In [ ]:
shex_jinja = """
{%- for prefix, uri in namespaces.items() %}
PREFIX {{prefix}}: <{{uri}}>
{%- endfor %}
BASE <http://purl.org/yama/example/mybook/v021/#>

<{{descriptionSet.descSetID}}> {
  {%- for description in descriptionSet.descriptions %}
  {{description.property}} @<{{description.descID}}>;
  {%- endfor %}
}

{%- for description in descriptions %}
<{{description.descID}}> {
  {%- for statement in description.statements%}
  {{statement.property}} @<{{statement.statementID}}> { {{statement.minOccur}},{{statement.maxOccur}} };
  {%- endfor %}
}
{%- endfor %}

{%- for statement in statements %}
<{{statement.statementID}}> {
    {{statement.property}}{% if not statement.value %} {{ statement.type }} {% endif %} {{statement.value.dataType}} ;
}
{%- endfor %}

"""

### 5. Rendering the ShExC Template with MAP and Jinja2

In [ ]:
from jinja2 import Template
template = Template(shex_jinja)
shexc_output = template.render(ap)
print(shexc_output)

### 6. Loading the PyShExC library

In [ ]:
import pyshexc
from pyshexc.parser_impl.generate_shexj import parse

### 7. Converting ShExC to ShExJ

In [ ]:
import json
json_ap=parse(shexc_output)._as_json
parsed = json.loads(json_ap)
print(json.dumps(parsed, indent=4, sort_keys=False))